<a href="https://colab.research.google.com/github/Amanda9805/Detecting-Machine-Generated-Texts/blob/train-model/COS_760_Project_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade datasets fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [ ]:
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

In [ ]:
import pandas as pd
import re
import nltk
from huggingface_hub import login
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
from datasets import load_dataset
import numpy as np
import torch


#shona_data = load_dataset("DigitalUmuganda/AfriVoice", "sn", streaming=True, split="train[:10%]")

# english_data = load_dataset("oscar-corpus/OSCAR-2201", language="en", streaming=True)

# num_samples = 500
# samples = []
# for i, example in enumerate(english_data["train"]):
#     if i >= num_samples:
#         break
#     samples.append(example['text'])

# eng_df = pd.DataFrame(samples, columns=['text'])
# eng_df.head()

zulu_data = load_dataset("dsfsi/vukuzenzele-monolingual", "zul")

def reformatData(dataDict):
  langList = []
  for index, row in dataDict.iterrows():
    # Access data using column names from the row Series
    if 'text' in row and row['text']:
        langList.append({
        'text': row.get('text', ''),
        'author': row.get('author', ''),
        'title': row.get('title', ''),
        'language': 'zul'
              })

  return langList

human_data = reformatData(zulu_data['train'].to_pandas())

zul_df = pd.DataFrame(human_data)
zul_df['cleaned_text'] = zul_df['text'].apply(clean_text)
zul_df['tokens'] = zul_df['cleaned_text'].apply(word_tokenize)
zul_df['label'] = 0
zul_df.head()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/312k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

eval-00000-of-00001.parquet:   0%|          | 0.00/74.0k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/129 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/28 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/28 [00:00<?, ? examples/s]

,text,author,title,language,cleaned_text,tokens,label
0,Ngokusho kukaNgqongqoshe Wezempilo uDkt. Joe P...,Allison Cooper\n,Zivikele kwi-COVID-19 ngaphambi kobusika \n,zul,ngokusho kukangqongqoshe wezempilo udkt joe ph...,"[ngokusho, kukangqongqoshe, wezempilo, udkt, j...",0
1,Abafundi BakaMa tikuletsheni bakulonyaka kumel...,Dale Hes \n,Kwethulwe ngokusemthethweni uhlelo lweWoza Ma...,zul,abafundi bakama tikuletsheni bakulonyaka kumel...,"[abafundi, bakama, tikuletsheni, bakulonyaka, ...",0
2,Lolu hlelo luhlose ukuthuthukisa ikhono labant...,Vukuzenzele Unnamed\n,Uhlelo lokudla esikoleni luqinisekisa ukuthi a...,zul,lolu hlelo luhlose ukuthuthukisa ikhono labant...,"[lolu, hlelo, luhlose, ukuthuthukisa, ikhono, ...",0
3,hulumeni eNingizimu Afrika yonkana abakwazi uk...,More Matshediso\n,Izisu ezigcwele ziholela emiphumeleni engcono\n,zul,hulumeni eningizimu afrika yonkana abakwazi uk...,"[hulumeni, eningizimu, afrika, yonkana, abakwa...",0
4,Kamuva nje bengivakashele eThekwini KwaZulu-Na...,Vukuzenzele Unnamed\n,Ukwakha kabusha impilo nendlela yokuphila emuv...,zul,kamuva nje bengivakashele ethekwini kwazulunat...,"[kamuva, nje, bengivakashele, ethekwini, kwazu...",0


In [ ]:
# eng_df['cleaned_text'] = eng_df['text'].apply(clean_text)
# eng_df['tokens'] = eng_df['cleaned_text'].apply(word_tokenize)
# print(eng_df.head())

zul_df['cleaned_text'] = zul_df['text'].apply(clean_text)
zul_df['tokens'] = zul_df['cleaned_text'].apply(word_tokenize)
zul_df.head()

In [ ]:
from transformers import pipeline
import json

en_generator = pipeline('text-generation', model='gpt2')


english_prompts = [
    "Explain the significance of lobola in Southern Africa",
    "Write a short dialogue between two friends in Johannesburg",
    "Describe linguistic features that make isiZulu agglutinative"
]

zulu_prompts = [
    "Chaza ngokubaluleka kwesiko lwelobola eNingizimu Afrika",
    "Bhala inkulumo emfushane phakathi kwabangani ababili eGoli",
    "Landela indaba yamaZulu ngokomlando",
    "Chaza ngamasiko amasha eZulu eskhathini samanje",
    "Bhala inganekwane ethi 'UNogwaja noFudu'"
]

# def generate_with_prompts(prompts, generator, language, samples_per_prompt=3):
#     data = []
#     for prompt in prompts:
#         for _ in range(samples_per_prompt):
#             output = generator(prompt, max_length=100, do_sample=True, temperature=0.7)
#             data.append({
#                 'prompt': prompt,
#                 'text': output[0]['generated_text'],
#                 'label': 'machine',
#                 'language': language,
#                 'prompt_type': 'cultural' if "tsika" in prompt else 'linguistic'  # Tag for analysis
#             })
#     return pd.DataFrame(data)


# eng_mg_df = generate_with_prompts(english_prompts, en_generator, 'English')
# eng_mg_df['cleaned_text'] = eng_mg_df['text'].apply(clean_text)
# eng_mg_df['tokens'] = eng_mg_df['cleaned_text'].apply(word_tokenize)
# print(eng_mg_df['text'].iloc[0][:300])

def load_jsonl_data(file_path):
    data = []
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                line = line.strip()
                if line:  # Skip empty lines
                    try:
                        data.append(json.loads(line))
                    except json.JSONDecodeError as e:
                        print(f"Error parsing JSON line: {e}")
                        continue
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return []

    return data

zul_mg_df = pd.DataFrame(load_jsonl_data('zulu_mg_text.jsonl'))

#zul_mg_df = pd.DataFrame(data)
machine_data =reformatData(zul_mg_df)
zul_mg_df = pd.DataFrame(machine_data)
zul_mg_df['text'] = zul_mg_df['text'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
zul_mg_df['language'] = 'zul'
zul_mg_df['cleaned_text'] = zul_mg_df['text'].apply(clean_text)
zul_mg_df['tokens'] = zul_mg_df['cleaned_text'].apply(word_tokenize)
zul_mg_df['label'] = 1

zul_mg_df.head()

COMBINE THE DATASETS
-------------------------

In [ ]:
# Combine both datasets, this will be bad for us if we have more than 70% differnce in data length
# Combine both datasets
all_texts = pd.concat([zul_df, zul_mg_df], ignore_index=True)
all_texts_shuffled = all_texts.sample(frac=1).reset_index(drop=True)
print(f"Total texts after combining: {len(all_texts)}")
print(f"Human-written texts: {len(zul_df)}")
print(f"Machine-generated texts: {len(zul_mg_df)}")

# Create a DataFrame for easier analysis
combined_df = pd.DataFrame(all_texts_shuffled)
combined_df.head()

In [ ]:
# Create a balanced dataset with equal samples from each class
# we can choose to use this of the combined one
def create_balanced_dataset(df, target_size_per_class=None):
    class_counts = df['label'].value_counts()
    min_class_size = class_counts.min()
    if target_size_per_class:
        sample_size = min(target_size_per_class, min_class_size)
    else:
        sample_size = min_class_size
    balanced_df = df.groupby('label').sample(n=sample_size, random_state=42)
    return balanced_df.reset_index(drop=True)

# Create balanced dataset, because we need the same number of samples for each class
balanced_df = create_balanced_dataset(combined_df)
print(f"\nBalanced dataset created with {len(balanced_df)} samples")
print(f"Label distribution in balanced dataset: \n{balanced_df['label'].value_counts()}")

# Save balanced dataset
balanced_output_path = 'balanced_zulu_texts.csv'
balanced_df.to_csv(balanced_output_path, index=False, encoding='utf-8')
print(f"Balanced dataset saved to: {balanced_output_path}")
balanced_df.head()

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
from torch.utils.data import Dataset, DataLoader
from transformers import (
    XLMRobertaTokenizer,
    XLMRobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

class ZuluTextDataset(Dataset):
    """Custom Dataset class for Zulu text classification"""

    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        # Tokenize the text
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

class ZuluTextClassifier:
    """XLM-RoBERTa classifier for Zulu human vs machine text detection"""

    def __init__(self, model_name='xlm-roberta-base', max_length=512):
        self.model_name = model_name
        self.max_length = max_length
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Initialize tokenizer and model
        self.tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
        self.model = XLMRobertaForSequenceClassification.from_pretrained(
            model_name,
            num_labels=2
        )
        self.model.to(self.device)

        print(f"Model initialized on {self.device}")
        print(f"Model parameters: {sum(p.numel() for p in self.model.parameters()):,}")

    def prepare_data(self, df, text_column='cleaned_text', label_column='label', test_size=0.2):
        """Prepare and split the data"""
        # Filter for Zulu language if language column exists
        if 'language' in df.columns:
            df_zulu = df[df['language'].str.lower().isin(['zul'])].copy()
            print(f"Filtered {len(df_zulu)} Zulu samples from {len(df)} total samples")
        else:
            df_zulu = df.copy()
            print(f"Using all {len(df_zulu)} samples (no language filtering)")

        if len(df_zulu) == 0:
            raise ValueError("No Zulu samples found in the dataset")

        # Extract texts and labels
        texts = df_zulu[text_column].tolist()
        labels = df_zulu[label_column].tolist()

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            texts, labels,
            test_size=test_size,
            random_state=42,
            stratify=labels
        )

        print(f"Training samples: {len(X_train)}")
        print(f"Test samples: {len(X_test)}")
        print(f"Label distribution in training: {np.bincount(y_train)}")
        print(f"Label distribution in test: {np.bincount(y_test)}")

        return X_train, X_test, y_train, y_test

    def create_datasets(self, X_train, X_test, y_train, y_test):
        """Create PyTorch datasets"""
        train_dataset = ZuluTextDataset(X_train, y_train, self.tokenizer, self.max_length)
        test_dataset = ZuluTextDataset(X_test, y_test, self.tokenizer, self.max_length)

        return train_dataset, test_dataset

    def train(self, train_dataset, eval_dataset, output_dir='./zulu_classifier_results'):
        """Train the model"""
        # Training arguments
        training_args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=5,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            warmup_steps=100,
            weight_decay=0.01,
            logging_dir=f'{output_dir}/logs',
            logging_steps=10,
            eval_strategy="steps",
            eval_steps=50,
            save_strategy="steps",
            save_steps=50,
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
            greater_is_better=False,
            save_total_limit=2,
            report_to=None,  # Disable wandb
            seed=42,
        )

        # Initialize trainer
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
        )

        # Train the model
        print("Starting training...")
        trainer.train()

        # Save the model
        trainer.save_model()
        self.tokenizer.save_pretrained(output_dir)

        return trainer

    def evaluate(self, test_dataset, trainer=None):
        """Evaluate the model"""
        if trainer is None:
            # Create a new trainer for evaluation
            trainer = Trainer(model=self.model)

        # Get predictions
        predictions = trainer.predict(test_dataset)
        y_pred = np.argmax(predictions.predictions, axis=1)
        y_true = predictions.label_ids

        # Calculate metrics
        accuracy = accuracy_score(y_true, y_pred)
        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

        # Per-class metrics
        precision_per_class, recall_per_class, f1_per_class, support = precision_recall_fscore_support(
            y_true, y_pred, average=None
        )

        results = {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'precision_per_class': precision_per_class,
            'recall_per_class': recall_per_class,
            'f1_per_class': f1_per_class,
            'support': support,
            'y_true': y_true,
            'y_pred': y_pred
        }

        return results

    def print_evaluation_results(self, results):
        """Print detailed evaluation results"""
        print("\n" + "="*50)
        print("EVALUATION RESULTS")
        print("="*50)

        print(f"Overall Accuracy: {results['accuracy']:.4f}")
        print(f"Weighted Precision: {results['precision']:.4f}")
        print(f"Weighted Recall: {results['recall']:.4f}")
        print(f"Weighted F1-Score: {results['f1']:.4f}")

        print("\nPer-Class Results:")
        class_names = ['Human', 'Machine']
        for i in range(len(class_names)):
            print(f"{class_names[i]}:")
            print(f"  Precision: {results['precision_per_class'][i]:.4f}")
            print(f"  Recall: {results['recall_per_class'][i]:.4f}")
            print(f"  F1-Score: {results['f1_per_class'][i]:.4f}")
            print(f"  Support: {results['support'][i]}")

        print("\nClassification Report:")
        print(classification_report(results['y_true'], results['y_pred'],
                                  target_names=class_names))



def main():
    """Main function to run the Zulu text classification"""

    df = balanced_df

    # Initialize classifier
    classifier = ZuluTextClassifier()

    # Prepare data
    X_train, X_test, y_train, y_test = classifier.prepare_data(df)

    # Create datasets
    train_dataset, test_dataset = classifier.create_datasets(X_train, X_test, y_train, y_test)

    # Train model
    trainer = classifier.train(train_dataset, test_dataset)

    # Evaluate
    results = classifier.evaluate(test_dataset, trainer)
    classifier.print_evaluation_results(results)


if __name__ == "__main__":
    main()
